# Data cleaning - American dream

In [1]:
#Modules
import pandas as pd
import sqlalchemy
import mysql.connector


import sys
sys.path.insert(0, "/home/apprenant/PycharmProjects/American_dream")


from src.d00_utils.mysql_func import mysql_connect, save_to_mysql
connect = mysql_connect()

## 1 - Importation des données

In [52]:
df1 = pd.read_sql("survey_1",con=connect)

In [67]:
dfk = pd.read_sql("survey_k",con=connect)

## 2 - Sélection des colonnes

### Database 1

In [4]:
print(df1.head())
df1.shape

   index  Survey Year           Timestamp  SalaryUSD        Country  \
0      0         2020 2020-01-04 18:50:34   115000.0  United States   
1      1         2020 2020-01-04 10:43:02   100000.0  United States   
2      2         2020 2020-01-04 09:51:46   100000.0          Spain   
3      3         2020 2020-01-04 01:08:54    70000.0  United States   
4      4         2020 2020-01-03 15:28:54   110000.0  United States   

  PostalCode       PrimaryDatabase  YearsWithThisDatabase  \
0         03  Microsoft SQL Server                     15   
1       None                 Other                      6   
2      28046  Microsoft SQL Server                      2   
3      94133  Microsoft SQL Server                      3   
4      95354                Oracle                     30   

                                      OtherDatabases    EmploymentStatus  ...  \
0  Microsoft SQL Server, MongoDB, Azure SQL DB (a...  Full time employee  ...   
1                                      MySQL

(8627, 32)

In [5]:
print(df1.columns)

Index(['index', 'Survey Year', 'Timestamp', 'SalaryUSD', 'Country',
       'PostalCode', 'PrimaryDatabase', 'YearsWithThisDatabase',
       'OtherDatabases', 'EmploymentStatus', 'JobTitle', 'ManageStaff',
       'YearsWithThisTypeOfJob', 'HowManyCompanies', 'OtherPeopleOnYourTeam',
       'CompanyEmployeesOverall', 'DatabaseServers', 'Education',
       'EducationIsComputerRelated', 'Certifications', 'HoursWorkedPerWeek',
       'TelecommuteDaysPerWeek', 'NewestVersionInProduction',
       'OldestVersionInProduction', 'PopulationOfLargestCityWithin20Miles',
       'EmploymentSector', 'LookingForAnotherJob', 'CareerPlansThisYear',
       'Gender', 'OtherJobDuties', 'KindsOfTasksPerformed', 'Counter'],
      dtype='object')


J'ai choisi les colonnes qui me semblaient utiles pour la visualisation des données.

In [53]:
df1=df1[['SalaryUSD', 'Country', 'PostalCode', 'EmploymentStatus', 'JobTitle', 'ManageStaff', 'YearsWithThisTypeOfJob', 'HowManyCompanies', 'OtherPeopleOnYourTeam', 'CompanyEmployeesOverall', 'EmploymentSector', 'LookingForAnotherJob', 'CareerPlansThisYear', 'Gender']]

### Database 2

In [68]:
print(dfk.head())
dfk.shape

   index  Unnamed: 0                                          Job Title  \
0      0           0  Data Analyst, Center on Immigration and Justic...   
1      1           1                               Quality Data Analyst   
2      2           2  Senior Data Analyst, Insights & Analytics Team...   
3      3           3                                       Data Analyst   
4      4           4                             Reporting Data Analyst   

              Salary Estimate  \
0  $37K-$66K (Glassdoor est.)   
1  $37K-$66K (Glassdoor est.)   
2  $37K-$66K (Glassdoor est.)   
3  $37K-$66K (Glassdoor est.)   
4  $37K-$66K (Glassdoor est.)   

                                     Job Description  Rating  \
0  Are you eager to roll up your sleeves and harn...     3.2   
1  Overview\n\nProvides analytical and technical ...     3.8   
2  We’re looking for a Senior Data Analyst who ha...     3.4   
3  Requisition NumberRR-0001939\nRemote:Yes\nWe c...     4.1   
4  ABOUT FANDUEL GROUP\n\nFanD

(2253, 17)

In [69]:
print(dfk.columns)

Index(['index', 'Unnamed: 0', 'Job Title', 'Salary Estimate',
       'Job Description', 'Rating', 'Company Name', 'Location', 'Headquarters',
       'Size', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue',
       'Competitors', 'Easy Apply'],
      dtype='object')


J'ai ensuite choisi les colonnes qui semblaient utiles pour la suite.

In [70]:
dfk=dfk[['Job Title', 'Salary Estimate', 'Company Name', 'Location', 'Size', 'Sector']]

## 3 - Valeurs manquantes

### Database 1

In [54]:
missing_values_count = df1.isnull().sum()
print(missing_values_count)

SalaryUSD                     0
Country                       0
PostalCode                 1390
EmploymentStatus              0
JobTitle                      0
ManageStaff                   0
YearsWithThisTypeOfJob        0
HowManyCompanies              0
OtherPeopleOnYourTeam         0
CompanyEmployeesOverall       0
EmploymentSector              0
LookingForAnotherJob          0
CareerPlansThisYear           0
Gender                        0
dtype: int64


Il manque 1390 valeurs pour PostalCode. On ne peut pas assigner de valeur de code postal arbitraire dans cette colonne. Je garde donc les lignes où les valeurs sont manquantes, mais je garde en mémoire qu'il manque des valeurs si je travaille sur les codes postaux.

### Database 2

In [71]:
print(dfk.isnull().sum())

Job Title          0
Salary Estimate    0
Company Name       1
Location           0
Size               0
Sector             0
dtype: int64


Il y a une seule valeur manquante pour 'Company Name'.
J'ai choisi de la remplacer par 'Unknown'

In [74]:
dfk.loc[dfk['Company Name'].isnull(), "Company Name"] = "Unknown"

In [75]:
print(dfk.isnull().sum())

Job Title          0
Salary Estimate    0
Company Name       0
Location           0
Size               0
Sector             0
dtype: int64


## 4 - Colonnes à transformer en datetime

### Database 1

In [55]:
print(df1.dtypes)

SalaryUSD                  float64
Country                     object
PostalCode                  object
EmploymentStatus            object
JobTitle                    object
ManageStaff                 object
YearsWithThisTypeOfJob       int64
HowManyCompanies            object
OtherPeopleOnYourTeam       object
CompanyEmployeesOverall     object
EmploymentSector            object
LookingForAnotherJob        object
CareerPlansThisYear         object
Gender                      object
dtype: object


Aucune de mes colonnes ne correspond à une date.

### Database 2

In [76]:
print(dfk.dtypes)

Job Title          object
Salary Estimate    object
Company Name       object
Location           object
Size               object
Sector             object
dtype: object


Aucune de mes colonnes ne correspond à une date.

## 5 - Doublons

### Database 1

In [56]:
print(df1.duplicated().value_counts())

False    8558
True       69
dtype: int64


Il y a 69 doublons dans notre jeu de données.
J'ai choisi de les supprimer.

In [57]:
df1=df1.drop_duplicates()

In [58]:
print(df1.shape)

(8558, 14)


### Database 2

In [77]:
print(dfk.duplicated().value_counts())

False    2253
dtype: int64


Cette database ne semble pas avoir de doublons.

## 6 - Vérifier la consistence des données

### Database 1

#### get all the unique values in the 'Country' column

In [59]:
countries = df1['Country'].unique()
print(len(countries))

93


In [15]:
countries.sort()
print(countries)

['Albania' 'Anguilla' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Bahrain' 'Belarus' 'Belgium' 'Bermuda' 'Bolivia' 'Brazil' 'Bulgaria'
 'Canada' 'Cayman Islands' 'China' 'Colombia' 'Costa Rica' 'Croatia'
 'Czech Republic' 'Denmark' 'Dominican Republic' 'Ecuador' 'El Salvador'
 'Estonia' 'Finland' 'France' 'Georgia' 'Germany' 'Ghana' 'Greece'
 'Guatemala' 'Guernsey' 'Hong Kong' 'Hungary' 'Iceland' 'India'
 'Indonesia' 'Iran' 'Ireland' 'Israel' 'Italy' 'Jamaica' 'Jersey' 'Jordan'
 'Kenya' 'Latvia' 'Lithuania' 'Luxembourg' 'Macedonia' 'Malaysia'
 'Maldives' 'Malta' 'Mexico' 'Moldova' 'Nepal' 'Netherlands' 'New Zealand'
 'Nicaragua' 'Norway' 'Pakistan' 'Paraguay' 'Peru' 'Philippines' 'Poland'
 'Portugal' 'Puerto Rico' 'Qatar' 'Romania' 'Russia' 'Saudi Arabia'
 'Serbia and Montenegro' 'Singapore' 'Slovakia' 'Slovenia' 'South Africa'
 'Spain' 'Swaziland' 'Sweden' 'Switzerland' 'Syria' 'Taiwan' 'Thailand'
 'Turkey' 'Uganda' 'Ukraine' 'United Arab Emirates' 'United Kingdom'
 'United States' 

Les noms de pays ne présentent pas de faute de frappe.

#### get all the unique values in the 'JobTitle' column

In [60]:
job_titles = df1['JobTitle'].unique()
print(len(job_titles))

23


In [16]:
job_titles.sort()
print(job_titles)

['Analyst' 'Analytics consultant' 'Architect' 'Consultant' 'DBA'
 'DBA (Development Focus - tunes queries, indexes, does deployments)'
 'DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers)'
 'DBA (Production Focus - build & troubleshoot servers, HA/DR)'
 'DBA / BI Developer' 'Data Scientist' 'Database Specialist'
 'DevOps, Sr Software Engineer DBA' 'Developer: App code (C#, JS, etc)'
 'Developer: Business Intelligence (SSRS, PowerBI, etc)'
 'Developer: T-SQL' 'Engineer' 'Manager' 'Other'
 'Principal database engineer' 'Sales' 'Sr Consultant '
 'Systems Administrator' 'Technician ']


Il y a 23 job titles différents.

#### get all the unique values in 'HowManyCompanies' column

In [18]:
nb_companies = df1['HowManyCompanies'].unique()
print(len(nb_companies))

7


In [22]:
nb_companies.sort()
print(nb_companies)

["1 (this is the only company where I've had this kind of position)"
 '2 (I worked at another similar position elsewhere before this one)' '3'
 '4' '5' '6 or more' 'Not Asked']


Les valeurs pour HowManyCompanies sont classées dans 7 catégories. Il n'y a pas de catégorie redondante. 

#### get all the unique values in the 'OtherPeopleOnYourTeam' column

In [20]:
team = df1['OtherPeopleOnYourTeam'].unique()
print(len(team))

7


In [23]:
team.sort()
print(team)

['1' '2' '3' '4' '5' 'More than 5' 'None']


Les valeurs pour OtherPeopleOnYourTeam sont classées dans 7 catégories. Il n'y a pas de catégorie redondante. 

#### get all the unique values in the 'Gender' column

In [61]:
genders = df1['Gender'].unique()
print(len(genders))

32


Il semble y avoir beaucoup de réponses différentes dans la colonne 'Gender

In [62]:
genders.sort()
print(genders)

['Alien' 'Always On Cluster' 'Any human one' 'Apache Helicopter'
 'Attack Helicopter' 'Attack Helicopter (M)'
 'Attack Helicopter. (serious dude, gender questions?)'
 'Attack helicopter'
 'Attackhelicopter with rabbitears and afro-icelandic teeth' 'Cyborg'
 'Dalek' 'Database Administrator' 'Dragon' 'Female' 'Grayson Perry'
 'I am Batman' 'Lizzard' 'Male' 'Meat Popsicle ' 'Muppet'
 'Non-binary/third gender' 'Not Asked' 'Pastafarian' 'Prefer not to say'
 'Reptilian'
 'There are only two genders for people, either male or female. I am a male.'
 'This question is inappropriate. ' 'Vulcan' 'confused'
 'human  This is also my race.' 'mosquito ' 'toad frog']


In [63]:
print(df1['Gender'].value_counts())

Male                                                                          5039
Not Asked                                                                     2863
Female                                                                         553
Prefer not to say                                                               59
Non-binary/third gender                                                         15
Attack helicopter                                                                2
Attack Helicopter                                                                2
Apache Helicopter                                                                1
confused                                                                         1
Cyborg                                                                           1
This question is inappropriate.                                                  1
human  This is also my race.                                                     1
toad

Sur la totalité des réponses dans la colonne Genre, on préférerait pouvoir les classer en 5 catégories : Male, Female, Non-binary/third gender, Prefer not to say et Not Asked.
Il y a donc 29 réponses à modifier.

In [64]:
valid_genders = ["Male", "Female", "Not Asked", "Prefer not to say", "Non-binary/third gender"]
df1.loc[~df1['Gender'].isin(valid_genders), "Gender"] = "other"

In [65]:
print(df1['Gender'].value_counts())

Male                       5039
Not Asked                  2863
Female                      553
Prefer not to say            59
other                        29
Non-binary/third gender      15
Name: Gender, dtype: int64


### Database 2

In [79]:
titles = dfk['Job Title'].unique()
print(len(titles))

1272


In [80]:
print(len(dfk['Salary Estimate'].unique()))

90


## 7 - Recherche de valeurs aberrantes

### Database 1

In [31]:
print(df1.SalaryUSD.describe())

count    8.627000e+03
mean     9.394615e+04
std      5.998446e+04
min      0.000000e+00
25%      6.500000e+04
50%      9.000000e+04
75%      1.150000e+05
max      1.850000e+06
Name: SalaryUSD, dtype: float64


In [32]:
print(df1.YearsWithThisTypeOfJob.describe())

count    8627.000000
mean        7.544453
std         6.541186
min         0.000000
25%         3.000000
50%         5.000000
75%        10.000000
max        45.000000
Name: YearsWithThisTypeOfJob, dtype: float64


Il semble ne pas y avoir de valeurs aberrantes.

### Database 2

### Sauvegarde des databases dans mysql

In [66]:
save_to_mysql(db_connect=connect,df_to_save=df1,df_name='survey_1_v1.0')